In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [49]:
customers = pd.read_csv('data/customers.csv')

In [38]:
customers

,customer_id,became_member_on,gender,age,income
0,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN,118,NaN
1,0610b486422d4921ae7d2bf64640c50b,20170715,F,55,112000.0
2,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN,118,NaN
3,78afa995795e4d85b5d9ceeca43f5fef,20170509,F,75,100000.0
4,a03223e636434f42ac4c3df47e8bac43,20170804,NaN,118,NaN
...,...,...,...,...,...
16995,6d5f3a774f3d4714ab0c092238f3a1d7,20180604,F,45,54000.0
16996,2cb4f97358b841b9a9773a7aa05a9d77,20180713,M,61,72000.0
16997,01d26f638c274aa0b965d24cefe3183f,20170126,M,49,73000.0
16998,9dc1421481194dcd9400aec7c9ae6366,20160307,F,83,50000.0


In [50]:
non_null_gender = customers['gender'].notnull()
null_gender = customers['gender'].isnull()

In [51]:
customer_gender_is_null = customers[null_gender]
customers = customers[non_null_gender]

In [ ]:
customer_gender_is_null.sort_values(by= 'became_member_on', ascending = False).values.tolist()

In [52]:
customers['became_member_on'] = customers['became_member_on'].astype(str)

In [53]:
customers['became_member_on'] = customers['became_member_on'].str[:4] + '-' + customers['became_member_on'].str[4:6]+'-'+customers['became_member_on'].str[-2:]

In [ ]:
customers.values.tolist()

In [57]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customer_id       14825 non-null  object 
 1   became_member_on  14825 non-null  object 
 2   gender            14825 non-null  object 
 3   age               14825 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 694.9+ KB


Download the cleaned data as a new CSV file with: customers.to_csv('customers_cleaned.csv')

In [59]:
events = pd.read_csv('data/events.csv')

In [60]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  306534 non-null  object
 1   event        306534 non-null  object
 2   value        306534 non-null  object
 3   time         306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [91]:
events['value'].unique()

array(["{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}",
       "{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}",
       "{'offer id': '2906b810c7d4411798c6938adc9daaa5'}", ...,
       "{'amount': 685.07}", "{'amount': 405.04}", "{'amount': 476.33}"],
      dtype=object)

In [72]:
type_1_filter = events['value'] == "{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}"
events_type_1 = events[type_1_filter]

In [76]:
type_2_filter = events['value'] == "{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}"
events_type_2 = events[type_2_filter]

In [84]:
type_3_filter = events['value'] == "{'offer id': '2906b810c7d4411798c6938adc9daaa5'}"
events_type_3 = events[type_3_filter]
events_type_3.shape

(11750, 4)